In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_date, year

# Função para criar sessão Spark
def create_spark_session() -> SparkSession:
    spark = (
        SparkSession.builder
        .appName("ETL Silver - Match")
        .enableHiveSupport()
        .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
        .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
        .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
        .config("spark.hadoop.fs.s3minio.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
        .getOrCreate()
    )
    return spark

# Inicializa Spark
spark = create_spark_session()

# Configurações MinIO
minio_delta_options = {
    "fs.s3a.access.key": "admin",
    "fs.s3a.secret.key": "senhasegura",
    "fs.s3a.endpoint": "http://minio:9000",
    "fs.s3a.path.style.access": "true",
    "fs.s3a.connection.ssl.enabled": "false"
}

# Caminho Bronze
bronze_match_path = "s3a://bucket-bronze-zone/soccer/Match"

# Leitura do Delta Lake Bronze
df_bronze = (
    spark.read
    .format("delta")
    .options(**minio_delta_options)
    .load(bronze_match_path)
)

# Transformações
df_silver = (
    df_bronze
    # Remover colunas _airbyte_*
    .drop(*[c for c in df_bronze.columns if c.startswith("_airbyte")])
    # Converter 'date' de string para DateType com apenas data (YYYY-MM-DD)
    .withColumn("match_date", to_date(col("date")))
    .drop("date")
    # Criar coluna 'year' para facilitar análises e particionamentos
    .withColumn("year", year(col("match_date")))
)

# Validação: verificar dados transformados
print(f"Quantidade de registros após transformação: {df_silver.count()}")
df_silver.show(5)
df_silver.printSchema()

# Caminho Silver
silver_match_path = "s3a://bucket-silver-zone/soccer/Match"

# Escrita no Delta Lake Silver com particionamento por 'year'
(
    df_silver
    # Remover registros com valores nulos (Dropna na etapa de transformação dava overflow de memória)
    .dropna()
    .write
    .format("delta")
    .mode("overwrite")
    .partitionBy("year")  # Particionamento por 'year'
    .option("overwriteSchema", "true")
    .options(**minio_delta_options)
    .save(silver_match_path)
)

print("Tabela 'Match' transformada, particionada por 'year' e gravada com sucesso na camada Silver no MinIO!")

# Finaliza Spark
spark.stop()


Quantidade de registros após transformação: 25979
+---+----+----+----+----+----+----+----+----+----+---+---+----+----+---+----+----+----+----+----+---+----+----+----+----+----+----+----+----+----+-----+-----+-----+-----+-----+------+---------+------+-------+---------+----------+----------+----------+------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------